In [219]:
import numpy as np
import pandas as pd

In [220]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [221]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [222]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [223]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [224]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [225]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,male,45.5,0,0,28.5,S
1,2,male,23.0,0,0,13.0,S


In [226]:
y_train.head(2)

,Survived
0,0
1,0


In [227]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 39.1+ KB


In [228]:
# from sklearn.impute import SimpleImputer
# si_age = SimpleImputer()
# si_embarked = SimpleImputer(strategy='most_frequent')

In [229]:
# X_train['Age'] = si_age.fit_transform(X_train[['Age']]).ravel()
# X_train['Embarked'] = si_embarked.fit_transform(X_train[['Embarked']]).ravel()

# X_test['Age'] = si_age.transform(X_test[['Age']]).ravel()
# X_test['Embarked'] = si_embarked.transform(X_test[['Embarked']]).ravel()

In [231]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [232]:
# X_train_ohe = ohe.fit_transform(X_train[['Sex', 'Embarked']])
# X_test_ohe = ohe.transform(X_test[['Sex', 'Embarked']])

In [233]:
# X_train_ohe = pd.DataFrame(X_train_ohe, columns=ohe.get_feature_names_out())
# X_train = pd.concat([X_train.drop(columns=['Sex', 'Embarked']), X_train_ohe], axis=1)

# X_test_ohe = pd.DataFrame(X_test_ohe, columns=ohe.get_feature_names_out())
# X_test = pd.concat([X_test.drop(columns=['Sex', 'Embarked']), X_test_ohe], axis=1)

In [234]:
# from sklearn.tree import DecisionTreeClassifier
# dtc = DecisionTreeClassifier()
# dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [237]:
# y_pred = dtc.predict(X_test)

In [238]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, y_pred)

0.7932960893854749

In [250]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [246]:
transformer1 = ColumnTransformer([
    ('si_age', SimpleImputer(), [2]), #why index? because it returns a numpy array not dataframe
    ('si_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')

In [247]:
transformer2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough')

In [248]:
transformer3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0, 10))
], remainder='passthrough')

In [251]:
# transformer4 = SelectKBest(score_func=chi2, k=5)

In [252]:
transformer5 = DecisionTreeClassifier()

In [253]:
# pipe = Pipeline([
#     ('transformer1', transformer1),
#     ('transformer2', transformer2),
#     ('transformer3', transformer3),
#     ('transformer4', transformer4),
#     ('transformer5', transformer5)
# ])

#another way of making pipeline

In [268]:
pipe = make_pipeline(transformer1, transformer2, transformer3, transformer5) #similar to make_column_transformer

In [269]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('si_age', SimpleImputer(),
                                                  [2]),
                                                 ('si_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [270]:
pipe.score(X_test, y_test)

0.7932960893854749

In [271]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('si_age', SimpleImputer(), [2]),
                                 ('si_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [272]:
pipe.named_steps['columntransformer-1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('si_age', SimpleImputer(), [2]),
                                ('si_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [273]:
pipe.named_steps['columntransformer-1'].transformers_[0]

('si_age', SimpleImputer(), [2])

In [274]:
pipe.named_steps['columntransformer-1'].transformers_[0][1]

SimpleImputer()

In [275]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([0.55337079])